<a href="https://colab.research.google.com/github/VladSkopenko/DataScience_homeworks/blob/main/Hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import os
import zipfile
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

Підготуємо датасет для кожного з видів активності і розрахуємо time domain features. Потім об’єднаємо всі датасети в один за допомогою методу pd.concat()

In [9]:
with zipfile.ZipFile('homework.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/homework')

base_path = '/content/homework/data/idle'
file_names = os.listdir(base_path)


ds_idle=pd.DataFrame()
for file in file_names:
  df=pd.read_csv(f"homework/data/idle/{file}")
  td=pd.DataFrame(
      {
          "maximum_value_x": [df['accelerometer_X'].max()],
          "minimum_value_x": [df['accelerometer_X'].min()],
          "interquartile_range_x": [df['accelerometer_X'].quantile(0.75)-df['accelerometer_X'].quantile(0.25)],
          "maximum_value_y": [df['accelerometer_Y'].max()],
          "index_of_minimum_value_y": [df['accelerometer_Y'].idxmin()],
          "mean_of_absolute_deviation_y": [np.mean(np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean()))],
          "median_y": [df['accelerometer_Y'].median()],
          "skewness_y": df.skew(axis=0)['accelerometer_Y'],
          "standard_deviation_y": [df['accelerometer_Y'].std()],
          "root_mean_square_error_y": [np.sqrt(np.mean(df['accelerometer_Y'])**2)],
          "skewness_z": df.skew(axis=0)['accelerometer_Z'],
          "id":['idle']

      }
)

  ds_idle=pd.concat([ds_idle,td]).reset_index(drop=True)
ds_idle

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,0.248997,-1.283292,0.276531,2.954445,1,0.440597,2.317588,-0.645705,0.540720,2.198516,-0.716329,idle
1,7.637504,-3.825934,1.315613,5.276821,27,1.412941,1.795651,0.045742,1.692111,1.426306,-3.923645,idle
2,2.518700,-1.283292,0.593762,4.046201,11,0.603466,2.501941,0.066132,0.757724,2.426284,-0.277219,idle
3,0.028730,-1.513135,0.533907,6.861782,14,0.449407,5.207389,0.679118,0.560803,5.336197,-0.200179,idle
4,3.739743,-1.072602,0.213083,9.916783,17,0.647807,4.292804,3.569480,1.202508,4.623523,-3.151390,idle
...,...,...,...,...,...,...,...,...,...,...,...,...
1034,7.637504,-3.825934,0.427365,7.661446,7,2.495812,5.154717,-0.782242,2.948666,4.046361,-2.747018,idle
1035,8.135497,-4.587290,0.961272,4.046201,2,0.952115,2.477999,-1.740086,1.301708,2.145045,-0.810544,idle
1036,0.488417,-0.861913,0.361524,7.139510,8,0.479607,5.973534,-0.323780,0.607568,5.899792,0.614462,idle
1037,0.488417,-0.861913,0.366313,7.139510,28,0.532981,6.016629,-0.649293,0.685922,5.894365,1.483550,idle


In [10]:
file_names=os.listdir('homework/data/running')
ds_run=pd.DataFrame()
for file in file_names:
  df=pd.read_csv(f"homework/data/running/{file}")
  td=pd.DataFrame(
      {
          "maximum_value_x": [df['accelerometer_X'].max()],
          "minimum_value_x": [df['accelerometer_X'].min()],
          "interquartile_range_x": [df['accelerometer_X'].quantile(0.75)-df['accelerometer_X'].quantile(0.25)],
          "maximum_value_y": [df['accelerometer_Y'].max()],
          "index_of_minimum_value_y": [df['accelerometer_Y'].idxmin()],
          "mean_of_absolute_deviation_y": [np.mean(np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean()))],
          "median_y": [df['accelerometer_Y'].median()],
          "skewness_y": df.skew(axis=0)['accelerometer_Y'],
          "standard_deviation_y": [df['accelerometer_Y'].std()],
          "root_mean_square_error_y": [np.sqrt(np.mean(df['accelerometer_Y'])**2)],
          "skewness_z": df.skew(axis=0)['accelerometer_Z'],
          "id":['running']

      }
)

  ds_run=pd.concat([ds_run,td]).reset_index(drop=True)
ds_run

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,39.188293,-5.463568,8.380903,37.885850,8,10.879487,6.253655,0.420204,13.421598,8.302453,-0.021160,running
1,37.244200,-4.711789,8.433575,30.368053,9,9.631756,8.099584,-0.043404,12.133718,8.029514,0.619732,running
2,34.342430,-4.371812,10.817004,28.064833,17,9.342611,7.649474,-0.106549,11.502344,6.294676,1.475548,running
3,30.674513,-8.006210,12.698845,30.004135,27,13.684203,7.754819,-0.235061,16.473316,7.153396,-0.647871,running
4,23.060951,-6.124367,12.032061,27.686548,20,8.337089,6.497864,0.091494,10.797554,6.837042,0.270494,running
...,...,...,...,...,...,...,...,...,...,...,...,...
3403,28.107927,-3.892972,9.566033,31.138987,0,9.832890,7.084443,0.040780,11.849705,9.152554,0.695424,running
3404,23.319525,-9.232041,7.496246,28.060043,22,9.593343,6.476316,0.246759,11.054261,9.676246,0.647566,running
3405,13.522451,-20.556616,11.980586,30.454247,1,10.439677,8.159440,0.144826,12.862445,8.481220,-0.184968,running
3406,38.326380,-10.644621,12.563573,37.521930,28,11.169749,9.136273,0.127440,13.851550,8.545704,1.140064,running


In [11]:
file_names=os.listdir('homework/data/stairs')
ds_stairs=pd.DataFrame()
for file in file_names:
  df=pd.read_csv(f"homework/data/stairs/{file}")
  td=pd.DataFrame(
      {
          "maximum_value_x": [df['accelerometer_X'].max()],
          "minimum_value_x": [df['accelerometer_X'].min()],
          "interquartile_range_x": [df['accelerometer_X'].quantile(0.75)-df['accelerometer_X'].quantile(0.25)],
          "maximum_value_y": [df['accelerometer_Y'].max()],
          "index_of_minimum_value_y": [df['accelerometer_Y'].idxmin()],
          "mean_of_absolute_deviation_y": [np.mean(np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean()))],
          "median_y": [df['accelerometer_Y'].median()],
          "skewness_y": df.skew(axis=0)['accelerometer_Y'],
          "standard_deviation_y": [df['accelerometer_Y'].std()],
          "root_mean_square_error_y": [np.sqrt(np.mean(df['accelerometer_Y'])**2)],
          "skewness_z": df.skew(axis=0)['accelerometer_Z'],
          "id":['stairs']

      }
)

  ds_stairs=pd.concat([ds_stairs,td]).reset_index(drop=True)
ds_stairs

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,12.650962,-2.011129,3.190274,1.618480,29,4.028111,-8.135497,-1.178340,5.526441,9.210334,1.439123,stairs
1,5.272033,-7.426813,4.807557,1.345541,19,4.724643,-11.463438,-0.322637,6.191359,11.622094,-1.714293,stairs
2,2.959233,-7.805097,3.986346,-0.943315,18,2.873840,-8.384495,-0.702513,3.912117,8.851364,1.692299,stairs
3,3.945645,-7.805097,3.178303,-3.380613,28,2.731050,-7.922414,-1.098966,3.679735,8.497501,1.506401,stairs
4,9.141062,-7.939172,1.957260,4.994305,3,4.686144,-10.283096,0.577565,6.354383,9.688536,-1.393456,stairs
...,...,...,...,...,...,...,...,...,...,...,...,...
160,5.272033,-7.426813,4.871003,1.345541,23,4.588248,-11.314997,-0.339455,6.033176,11.779473,-1.384562,stairs
161,7.484275,-6.287173,4.696227,-4.065354,8,3.776135,-8.559271,-0.945304,4.799924,9.857886,-1.837929,stairs
162,7.771579,-6.287173,4.809952,-3.892972,15,4.841884,-9.947909,-0.683151,5.875393,11.617943,-1.445356,stairs
163,8.073248,-4.582502,3.848680,2.973599,19,3.314692,-7.153874,-0.280692,4.710145,7.161376,-0.473734,stairs


In [12]:
file_names=os.listdir('homework/data/walking')
ds_walking=pd.DataFrame()
for file in file_names:
  df=pd.read_csv(f"homework/data/walking/{file}")
  td=pd.DataFrame(
      {
          "maximum_value_x": [df['accelerometer_X'].max()],
          "minimum_value_x": [df['accelerometer_X'].min()],
          "interquartile_range_x": [df['accelerometer_X'].quantile(0.75)-df['accelerometer_X'].quantile(0.25)],
          "maximum_value_y": [df['accelerometer_Y'].max()],
          "index_of_minimum_value_y": [df['accelerometer_Y'].idxmin()],
          "mean_of_absolute_deviation_y": [np.mean(np.abs(df["accelerometer_Y"] - df["accelerometer_Y"].mean()))],
          "median_y": [df['accelerometer_Y'].median()],
          "skewness_y": df.skew(axis=0)['accelerometer_Y'],
          "standard_deviation_y": [df['accelerometer_Y'].std()],
          "root_mean_square_error_y": [np.sqrt(np.mean(df['accelerometer_Y'])**2)],
          "skewness_z": df.skew(axis=0)['accelerometer_Z'],
          "id":['walking']

      }
)

  ds_walking=pd.concat([ds_walking,td]).reset_index(drop=True)
ds_walking

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,3.122039,-9.313444,3.428497,-1.953669,1,2.989836,-9.528923,0.124810,3.845347,9.083920,2.069450,walking
1,2.370260,-14.695610,4.812345,7.824251,27,4.297656,-11.532870,0.413673,5.878172,9.820536,0.401795,walking
2,3.936068,-13.584701,2.811990,-2.489970,29,3.186363,-9.227253,-0.056773,3.959613,9.270828,-1.463731,walking
3,9.375694,-12.876017,5.008670,10.055647,15,6.299028,-10.752360,0.074251,8.712900,11.229444,-0.502114,walking
4,0.914585,-21.346703,7.912837,7.584831,18,5.448682,-11.880028,0.275742,7.361223,11.312922,-0.620971,walking
...,...,...,...,...,...,...,...,...,...,...,...,...
1845,3.615245,-8.595184,3.968389,3.500323,19,3.080220,-8.456320,0.968116,4.019453,8.319372,2.591073,walking
1846,16.874334,-6.186617,3.825934,0.373495,6,4.523403,-10.177751,0.076542,5.643059,9.979671,-1.577995,walking
1847,0.981623,-12.775460,4.720169,-2.193089,13,3.320066,-10.910377,-0.120702,4.161547,10.611900,-0.543922,walking
1848,8.686164,-16.266207,7.097611,14.250289,28,6.401776,-12.521675,0.978604,9.006088,11.027693,-1.284272,walking


In [13]:
data=pd.concat([ds_idle, ds_run, ds_stairs, ds_walking]).reset_index(drop=True)
data

,maximum_value_x,minimum_value_x,interquartile_range_x,maximum_value_y,index_of_minimum_value_y,mean_of_absolute_deviation_y,median_y,skewness_y,standard_deviation_y,root_mean_square_error_y,skewness_z,id
0,0.248997,-1.283292,0.276531,2.954445,1,0.440597,2.317588,-0.645705,0.540720,2.198516,-0.716329,idle
1,7.637504,-3.825934,1.315613,5.276821,27,1.412941,1.795651,0.045742,1.692111,1.426306,-3.923645,idle
2,2.518700,-1.283292,0.593762,4.046201,11,0.603466,2.501941,0.066132,0.757724,2.426284,-0.277219,idle
3,0.028730,-1.513135,0.533907,6.861782,14,0.449407,5.207389,0.679118,0.560803,5.336197,-0.200179,idle
4,3.739743,-1.072602,0.213083,9.916783,17,0.647807,4.292804,3.569480,1.202508,4.623523,-3.151390,idle
...,...,...,...,...,...,...,...,...,...,...,...,...
6457,3.615245,-8.595184,3.968389,3.500323,19,3.080220,-8.456320,0.968116,4.019453,8.319372,2.591073,walking
6458,16.874334,-6.186617,3.825934,0.373495,6,4.523403,-10.177751,0.076542,5.643059,9.979671,-1.577995,walking
6459,0.981623,-12.775460,4.720169,-2.193089,13,3.320066,-10.910377,-0.120702,4.161547,10.611900,-0.543922,walking
6460,8.686164,-16.266207,7.097611,14.250289,28,6.401776,-12.521675,0.978604,9.006088,11.027693,-1.284272,walking


Застосуємо алгоритми SVM та випадковий ліс з бібліотеки scikit-learn до нашого датасету. Метрики оцінки, такі як точність (Аccuracy) та F1-показник (F1 Score), допомагають зрозуміти, яка модель працює краще для даного датасету. Confusion matrix допоможе побачити, які класи добре або погано класифікуються кожною з моделей. Ми також використовуємо GridSearchCV для пошуку найкращих значень гіперпараметрів C для SVM та max_depth для випадкового лісу. Крос-валідація з параметром cv=5 допомагає уникнути перенавчання та підвищити надійність результатів.

In [14]:
X = data.drop("id", axis=1)
y = data["id"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_param_grid = {'C': [0.1, 1.0, 10.0, 100.0]}
svm_model = SVC()
svm_grid_search = GridSearchCV(svm_model, svm_param_grid, cv=5)
svm_grid_search.fit(X_train, y_train)

rf_param_grid = {'max_depth': [None, 10, 20, 30]}
rf_model = RandomForestClassifier(random_state=42)
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

best_svm_model = svm_grid_search.best_estimator_
best_rf_model = rf_grid_search.best_estimator_

svm_preds = best_svm_model.predict(X_test)
rf_preds = best_rf_model.predict(X_test)

svm_accuracy = accuracy_score(y_test, svm_preds)
rf_accuracy = accuracy_score(y_test, rf_preds)

svm_f1 = f1_score(y_test, svm_preds, average='weighted')
rf_f1 = f1_score(y_test, rf_preds, average='weighted')

svm_confusion_matrix = confusion_matrix(y_test, svm_preds)
rf_confusion_matrix = confusion_matrix(y_test, rf_preds)

print("Best SVM Model Parameters:", svm_grid_search.best_params_)
print("Best Random Forest Model Parameters:", rf_grid_search.best_params_)

print("\nSVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("\nSVM F1 Score:", svm_f1)
print("Random Forest F1 Score:", rf_f1)

print("\nSVM Confusion Matrix:")
print(svm_confusion_matrix)

print("\nRandom Forest Confusion Matrix:")
print(rf_confusion_matrix)

Best SVM Model Parameters: {'C': 100.0}
Best Random Forest Model Parameters: {'max_depth': None}

SVM Accuracy: 0.991492652745553
Random Forest Accuracy: 0.9984532095901005

SVM F1 Score: 0.9914265561220514
Random Forest F1 Score: 0.9984287193679718

SVM Confusion Matrix:
[[220   0   0   0]
 [  0 689   0   0]
 [  0   0  24   6]
 [  0   0   5 349]]

Random Forest Confusion Matrix:
[[220   0   0   0]
 [  0 689   0   0]
 [  0   0  28   2]
 [  0   0   0 354]]


Як бачимо з отриманих результатів, обидва алгоритми досить непогано справилися із задачею класифікації видів активності та все ж Random Forest виявився на один відсоток точнішим за SVM. До того ж, як показує Confusion Matrix, обидві моделі мали певні проблеми з класифікацією такого виду активності як stairs. Але якщо SVM виявив 15 помилкових класифікацій, то випадковий ліс лише 3. Також SVM один раз помилився при класифікації walking. Загалом, результати показують, що випадковий ліс показує кращі результати в порівнянні з SVM на цьому конкретному датасеті, оскільки має вищі значення точності та F1-показника, а також меншу кількість помилкових класифікацій.